In [ ]:
%cd /content/ai-competition-baselines
!python -m src.stage3_localization.weak_cam.infer \
  --config configs/stage3_weak_cam.yaml \
  --data_root "$DATA_ROOT"

In [ ]:
%cd /content/ai-competition-baselines
!python -m src.stage3_localization.weak_cam.eval_pointing_game \
  --config configs/stage3_weak_cam.yaml \
  --data_root "$DATA_ROOT" \
  --radius 15
